In [95]:
import pandas as pd
import tensorflow as tf
import keras as ks
import numpy as np

from keras.layers import Dense, LSTM, Input
from keras.callbacks import ModelCheckpoint

import matplotlib.pylab as plt

In [163]:
# Create new variables to be used in Keras and the CNN

# number of items to use for training
BATCH_SIZE = 20 

# Number of identifying classes 
#   WE have two, Bloom and no bloom 1/0
NUM_CLASSES = 2 

# number of times to repeat process
EPOCHS = 100

In [97]:
df = pd.read_csv('../../data/cleaned/site1_vineyard.csv')
df.head()

,Date (mm.dd.yyyy),Time 24hr,Temp C,Sp Cond (uS/cm),pH (mV),pH,Turbidity (NTU),Chlorophyll (ug/L),Chlorophyll RFU,ODOSat%,ODO (mg/L),BGA-Phycocyanin RFU
0,5/5/2017,0:00,15.02,1848,-100.1,8.36,16.84,4.4,1.3,90.2,9.04,0.4
1,5/5/2017,0:15,14.99,1847,-100.1,8.36,16.76,4.2,1.2,90.2,9.04,0.4
2,5/5/2017,0:30,14.96,1847,-100.1,8.36,16.82,4.3,1.3,90.1,9.04,0.4
3,5/5/2017,0:45,14.95,1848,-100.1,8.36,17.19,4.5,1.3,90.0,9.03,0.4
4,5/5/2017,1:00,14.92,1848,-100.0,8.36,16.85,4.5,1.3,89.8,9.02,0.4


In [98]:
target = df['BGA-Phycocyanin RFU'].apply(lambda x : x / 0.2334)
target
dataset = df.drop(columns=['Chlorophyll (ug/L)', 'Chlorophyll RFU'])
dataset['BGA (ug/L)'] = target
dataset.head(5)

,Date (mm.dd.yyyy),Time 24hr,Temp C,Sp Cond (uS/cm),pH (mV),pH,Turbidity (NTU),ODOSat%,ODO (mg/L),BGA-Phycocyanin RFU,BGA (ug/L)
0,5/5/2017,0:00,15.02,1848,-100.1,8.36,16.84,90.2,9.04,0.4,1.713796
1,5/5/2017,0:15,14.99,1847,-100.1,8.36,16.76,90.2,9.04,0.4,1.713796
2,5/5/2017,0:30,14.96,1847,-100.1,8.36,16.82,90.1,9.04,0.4,1.713796
3,5/5/2017,0:45,14.95,1848,-100.1,8.36,17.19,90.0,9.03,0.4,1.713796
4,5/5/2017,1:00,14.92,1848,-100.0,8.36,16.85,89.8,9.02,0.4,1.713796


In [99]:
from datetime import datetime

timestamp = dataset['Date (mm.dd.yyyy)'] + ' '+ dataset['Time 24hr']
timestamp = pd.to_datetime(timestamp)
dataset['Timestamp'] = timestamp
dataset.head()

,Date (mm.dd.yyyy),Time 24hr,Temp C,Sp Cond (uS/cm),pH (mV),pH,Turbidity (NTU),ODOSat%,ODO (mg/L),BGA-Phycocyanin RFU,BGA (ug/L),Timestamp
0,5/5/2017,0:00,15.02,1848,-100.1,8.36,16.84,90.2,9.04,0.4,1.713796,2017-05-05 00:00:00
1,5/5/2017,0:15,14.99,1847,-100.1,8.36,16.76,90.2,9.04,0.4,1.713796,2017-05-05 00:15:00
2,5/5/2017,0:30,14.96,1847,-100.1,8.36,16.82,90.1,9.04,0.4,1.713796,2017-05-05 00:30:00
3,5/5/2017,0:45,14.95,1848,-100.1,8.36,17.19,90.0,9.03,0.4,1.713796,2017-05-05 00:45:00
4,5/5/2017,1:00,14.92,1848,-100.0,8.36,16.85,89.8,9.02,0.4,1.713796,2017-05-05 01:00:00


In [100]:
# dont need data and time now that we have Timestamp. Lets remove them

dataset = dataset.drop(columns=['Date (mm.dd.yyyy)', 'Time 24hr'])
dataset.head(10)


,Temp C,Sp Cond (uS/cm),pH (mV),pH,Turbidity (NTU),ODOSat%,ODO (mg/L),BGA-Phycocyanin RFU,BGA (ug/L),Timestamp
0,15.02,1848,-100.1,8.36,16.84,90.2,9.04,0.4,1.713796,2017-05-05 00:00:00
1,14.99,1847,-100.1,8.36,16.76,90.2,9.04,0.4,1.713796,2017-05-05 00:15:00
2,14.96,1847,-100.1,8.36,16.82,90.1,9.04,0.4,1.713796,2017-05-05 00:30:00
3,14.95,1848,-100.1,8.36,17.19,90.0,9.03,0.4,1.713796,2017-05-05 00:45:00
4,14.92,1848,-100.0,8.36,16.85,89.8,9.02,0.4,1.713796,2017-05-05 01:00:00
5,14.92,1850,-100.1,8.36,16.43,89.8,9.02,0.4,1.713796,2017-05-05 01:15:00
6,14.90,1851,-100.1,8.36,16.35,89.7,9.01,0.4,1.713796,2017-05-05 01:30:00
7,14.88,1852,-100.0,8.36,16.40,89.6,9.00,0.4,1.713796,2017-05-05 01:45:00
8,14.84,1850,-99.9,8.36,16.82,89.4,8.99,0.4,1.713796,2017-05-05 02:00:00
9,14.83,1851,-100.0,8.36,16.50,89.4,8.99,0.4,1.713796,2017-05-05 02:15:00


In [101]:
target = dataset['BGA (ug/L)'].apply(lambda x: 1 if x > 20 else 0)
dataset['Bloom'] = target
dataset.dtypes

Temp C                        float64
Sp Cond (uS/cm)                 int64
pH (mV)                       float64
pH                            float64
Turbidity (NTU)               float64
ODOSat%                       float64
ODO (mg/L)                    float64
BGA-Phycocyanin RFU           float64
BGA (ug/L)                    float64
Timestamp              datetime64[ns]
Bloom                           int64
dtype: object

In [102]:
# lets try to normalize this now....
from sklearn.preprocessing import MinMaxScaler
dataset_columns = ['Temp C','Sp Cond (uS/cm)', 'pH (mV)','pH', 'Turbidity (NTU)', 'ODOSat%','ODO (mg/L)', 'Bloom']
scaler = MinMaxScaler()
ds_scaled = scaler.fit_transform(dataset[dataset_columns])
dataset = pd.DataFrame(ds_scaled,columns=dataset_columns)
dataset.describe()

C:\Users\mcneelypj\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,Temp C,Sp Cond (uS/cm),pH (mV),pH,Turbidity (NTU),ODOSat%,ODO (mg/L),Bloom
count,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000,18947.000000
mean,0.560516,0.893366,0.477889,0.541089,0.090355,0.165126,0.273546,0.005120
std,0.267413,0.070246,0.116241,0.113254,0.085879,0.093899,0.116220,0.071369
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.308195,0.853771,0.426166,0.461538,0.045893,0.117086,0.190000,0.000000
50%,0.640366,0.895882,0.492228,0.553846,0.065216,0.138870,0.264545,0.000000
75%,0.784656,0.956502,0.538860,0.592308,0.102407,0.177672,0.370909,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [103]:
'''
determines the window size for the daata set
@param dataset - The dataset to get windows for
@param window_size - the size of the window  
@param shift - the amout to shift the window
'''
def windows(dataset, window_size, shift):
    start = 0
    while start+window_size < dataset.shape[0]: 
        yield (int(start), int(start+window_size))
        # shift the window five blocks of time
        start += shift
        if start % 300 == 0:
            print('Window Segmentation {0:.2f}% done'.format(((start+window_size) / dataset.shape[0]) * 100 ))


'''

Segments the dataset based on the parameters that are passed in.
@param dataset - the dataset to segment into window
@param columns - the array of columns from the dataset to be looked at
@param window_size - the size of the window you would like to be looked at. Defualt is 10

'''
def segment_dataset(dataset, columns, target, window_size=10):    
    print('WINDOW SIZE',window_size)
    print('NUMBER OF COULUMNS',len(columns))
    segments = np.empty((0, window_size, len(columns)))
    labels = np.empty((0))
    count = 0
    for (start, end) in windows(dataset, window_size, 1):
        count+=1
        values = dataset[columns][start:end]
        if(values.shape[0] == window_size):
            segments = np.vstack([segments, np.stack([values])])
            # Takes the larger of the two variables if there are more than one. 
            # This makes it more likly to predict a bloom. Can be changed to iloc[0] to
            # be less likly to predict a bloom (more 0s in the label array)
            
            labels = np.append(labels, dataset[target][start:end].mode().iloc[-1])
        else:
            print("No more Windows available... Exiting")
            break
    return (segments, labels)

In [69]:
feature_columns = dataset_columns[:-1]
(segments, labels) = segment_dataset(dataset, feature_columns, 'Bloom', 5)
print('done')

WINDOW SIZE 5
NUMBER OF COULUMNS 7
Window Segmentation 1.61% done
Window Segmentation 3.19% done
Window Segmentation 4.78% done
Window Segmentation 6.36% done
Window Segmentation 7.94% done
Window Segmentation 9.53% done
Window Segmentation 11.11% done
Window Segmentation 12.69% done
Window Segmentation 14.28% done
Window Segmentation 15.86% done
Window Segmentation 17.44% done
Window Segmentation 19.03% done
Window Segmentation 20.61% done
Window Segmentation 22.19% done
Window Segmentation 23.78% done
Window Segmentation 25.36% done
Window Segmentation 26.94% done
Window Segmentation 28.53% done
Window Segmentation 30.11% done
Window Segmentation 31.69% done
Window Segmentation 33.28% done
Window Segmentation 34.86% done
Window Segmentation 36.44% done
Window Segmentation 38.03% done
Window Segmentation 39.61% done
Window Segmentation 41.19% done
Window Segmentation 42.78% done
Window Segmentation 44.36% done
Window Segmentation 45.94% done
Window Segmentation 47.53% done
Window Segm

In [105]:
print(dataset.shape)
target = dataset['Bloom']
features = dataset.drop(columns={'Bloom'})

(18947, 8)


In [71]:
#labels.shape

(18942,)

In [72]:
#labels

array([0., 0., 0., ..., 0., 0., 0.])

In [16]:
#labels.shape

(18938,)

In [74]:
#labels = labels.reshape(labels.shape[0],1)
#labels.shape

(18942, 1)

In [106]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

In [107]:
print("x_train shape:",x_train.shape)
print("x_test shape:",x_test.shape)
print("y_train shape:",y_train.shape)
print("y_test shape:",y_test.shape)

x_train shape: (13262, 7)
x_test shape: (5685, 7)
y_train shape: (13262,)
y_test shape: (5685,)


In [123]:
y_train_mod = ks.utils.to_categorical(y_train, NUM_CLASSES)
y_test_mod = ks.utils.to_categorical(y_test, NUM_CLASSES)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

## RNN 

In [109]:
from keras import backend as K

def precision(y_true, y_pred):
    '''
     Calculates the precision, a metric for multi-label classification of
     how many selected items are relevant.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """
     Recall metric.
     Only computes a batch-wise average of recall.
     Computes the recall, a metric for multi-label classification of
     how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [110]:
df.shape

(18947, 12)

In [265]:
use_dropout = True
hidden_size = 500
num_steps = 30

model = Sequential()

model.add(Embedding(x_train.shape[1],128,input_length=x_train.shape[1],dropout=0.2))
model.add(LSTM(hidden_size, dropout=0.2))
model.add(Dense(2,activation='softmax'))


C:\Users\mcneelypj\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  import sys


In [270]:
model.compile(loss=ks.losses.categorical_crossentropy,
              optimizer=ks.optimizers.Adam(lr=0.0001),
              metrics=[recall])

In [271]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 7, 128)            896       
_________________________________________________________________
lstm_51 (LSTM)               (None, 500)               1258000   
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 1002      
Total params: 1,259,898
Trainable params: 1,259,898
Non-trainable params: 0
_________________________________________________________________


In [272]:
print(x_train.shape)
print(y_train_mod.shape)
x_train_reshaped = x_train.values.reshape(x_train.shape[0],x_train.shape[1],1)
print(x_train_reshaped.shape)

(13262, 7)
(13262, 2)
(13262, 7, 1)


In [273]:
model.fit(x_train, 
          y_train_mod,
          batch_size=BATCH_SIZE,          
          epochs=EPOCHS)

Epoch 1/100
13262/13262 [==============================] - 65s 5ms/step - loss: 0.0315 - recall: 0.9952
Epoch 2/100
13262/13262 [==============================] - 57s 4ms/step - loss: 0.0310 - recall: 0.9952
Epoch 3/100
13262/13262 [==============================] - 65s 5ms/step - loss: 0.0313 - recall: 0.9952
Epoch 4/100
13262/13262 [==============================] - 63s 5ms/step - loss: 0.0313 - recall: 0.9952
Epoch 5/100
13262/13262 [==============================] - 62s 5ms/step - loss: 0.0309 - recall: 0.9952
Epoch 6/100
13262/13262 [==============================] - 59s 4ms/step - loss: 0.0314 - recall: 0.9952
Epoch 7/100
13262/13262 [==============================] - 55s 4ms/step - loss: 0.0312 - recall: 0.9952
Epoch 8/100
13262/13262 [==============================] - 54s 4ms/step - loss: 0.0310 - recall: 0.9952
Epoch 9/100
13262/13262 [==============================] - 55s 4ms/step - loss: 0.0311 - recall: 0.9952
Epoch 10/100
13262/13262 [==============================] - 54s 

13262/13262 [==============================] - 54s 4ms/step - loss: 0.0307 - recall: 0.9952
Epoch 80/100
13262/13262 [==============================] - 55s 4ms/step - loss: 0.0307 - recall: 0.9952
Epoch 81/100
13262/13262 [==============================] - 55s 4ms/step - loss: 0.0307 - recall: 0.9952
Epoch 82/100
13262/13262 [==============================] - 55s 4ms/step - loss: 0.0307 - recall: 0.9952
Epoch 83/100
13262/13262 [==============================] - 55s 4ms/step - loss: 0.0307 - recall: 0.9952
Epoch 84/100
13262/13262 [==============================] - 55s 4ms/step - loss: 0.0307 - recall: 0.9952
Epoch 85/100
13262/13262 [==============================] - 55s 4ms/step - loss: 0.0307 - recall: 0.9952
Epoch 86/100
13262/13262 [==============================] - 55s 4ms/step - loss: 0.0307 - recall: 0.9952
Epoch 87/100
13262/13262 [==============================] - 55s 4ms/step - loss: 0.0307 - recall: 0.9952
Epoch 88/100
13262/13262 [==============================] - 56s 4ms/

In [275]:
# score = model.evaluate(x_train, y_train_mod, verbose=0)
x_test_reshaped = x_test.values.reshape(x_test.shape[0],x_test.shape[1],1)
predictions = model.predict(x_test)

In [277]:
print(predictions)

[[0.9956938  0.00430618]
 [0.9956938  0.00430618]
 [0.9956938  0.00430618]
 ...
 [0.9956938  0.00430618]
 [0.9956938  0.00430618]
 [0.9956938  0.00430619]]
